In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import *


Bad key "text.kerning_factor" on line 4 in
/opt/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [74]:
data = pd.read_csv('../data/diamonds_train.csv')
data

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95
...,...,...,...,...,...,...,...,...,...,...
40450,1.34,Ideal,G,VS1,62.7,57.0,10070,7.10,7.04,4.43
40451,2.02,Good,F,SI2,57.1,60.0,12615,8.31,8.25,4.73
40452,1.01,Ideal,H,SI1,62.7,56.0,5457,6.37,6.42,4.01
40453,0.33,Ideal,J,VS1,61.9,54.3,456,4.45,4.47,2.76


In [137]:
data = data.loc[(data[['x','y','z']]!=0).all(axis=1)]

In [138]:
target = 'price'
cat_features = ['cut', 'color', 'clarity']
num_features = ['carat','depth','table']


X_train, X_test, y_train, y_test = train_test_split(data, data[target], test_size = 0.3, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42)


X_train[cat_features] = X_train.loc[:, cat_features].astype('category')
X_test[cat_features] = X_test.loc[:, cat_features].astype('category')
X_val[cat_features] = X_val.loc[:, cat_features].astype('category')

cat_df_train = pd.get_dummies(X_train[cat_features], drop_first = False)
num_df_train = X_train.loc[:, num_features]
X_train = pd.concat([cat_df_train, num_df_train], axis = 1)

cat_df_test = pd.get_dummies(X_test[cat_features], drop_first = False)
num_df_test = X_test.loc[:, num_features]
X_test = pd.concat([cat_df_test, num_df_test], axis = 1)

cat_df_val = pd.get_dummies(X_val[cat_features], drop_first = False)
num_df_val = X_val.loc[:, num_features]
X_val = pd.concat([cat_df_val, num_df_val], axis = 1)

features = list(cat_df_train.columns) + list(num_df_train.columns) 


In [139]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline
pipeline = [
    StandardScaler(),
    Normalizer(),
]

tr = make_pipeline(*pipeline)

Xpr = tr.fit_transform(X_train)
Xpr = pd.DataFrame(Xpr,columns=X_train.columns)
Xpr.head()

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,...,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,carat,depth,table
0,-0.045917,-0.083417,0.324279,-0.153997,-0.143120,-0.100466,0.559829,-0.122749,-0.134936,-0.112651,...,-0.048507,-0.150591,0.583533,-0.110241,-0.143580,-0.070619,-0.084601,0.005777,-0.137233,-0.173731
1,-0.041997,-0.076295,0.296590,-0.140848,-0.130899,-0.091887,-0.113748,-0.112268,-0.123414,0.565278,...,-0.044365,-0.137733,-0.109127,0.577640,-0.131320,-0.064589,-0.077377,0.259423,-0.041318,-0.158897
2,-0.045877,-0.083344,-0.214515,-0.153861,0.486047,-0.100377,-0.124257,-0.122641,0.515526,-0.112552,...,-0.048464,-0.150458,-0.119210,-0.110143,0.484488,-0.070556,-0.084527,0.116820,0.194005,0.063060
3,-0.036475,0.663020,-0.170555,-0.122331,-0.113690,-0.079807,-0.098793,-0.097508,0.409880,-0.089487,...,-0.038532,0.367269,-0.094780,-0.087572,-0.114056,-0.056097,-0.067205,0.088466,0.022617,0.332354
4,-0.038077,-0.069174,0.268909,-0.127703,-0.118682,-0.083311,-0.103132,0.470358,-0.111896,-0.093416,...,-0.040224,-0.124878,-0.098942,-0.091417,-0.119064,-0.058561,0.682450,-0.211805,0.008342,-0.242270


In [155]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

fit_params = {'early_stopping_rounds': 30,
             'eval_set': [(X_test, y_test)]}


model = lgb.LGBMRegressor()
param_dist = {'num_leaves': [35,40,45,50,55,60,65],
             'min_data_in_leaf': [10, 15, 20, 30, 40],
             'max_depth': [2, 4, 8, 16],
              'max_bin' : [100],
              'learning_rate' : [0.08]
            }
cv = 5
scoring_fit = 'neg_root_mean_squared_error' 
n_estimators = [16 ,32 ,64 ,128 ,254 ,512]

gs = RandomizedSearchCV(estimator = model,
                        param_distributions = param_dist,
                        cv = cv,
                        n_jobs = -1,
                        scoring = scoring_fit,
                         n_iter=50,
                       )


fit_model = gs.fit(X_train, y_train, **fit_params)

[1]	valid_0's l2: 1.35947e+07
Training until validation scores don't improve for 30 rounds
[2]	valid_0's l2: 1.1612e+07
[3]	valid_0's l2: 9.93533e+06
[4]	valid_0's l2: 8.50726e+06
[5]	valid_0's l2: 7.30453e+06
[6]	valid_0's l2: 6.27905e+06
[7]	valid_0's l2: 5.41172e+06
[8]	valid_0's l2: 4.67563e+06
[9]	valid_0's l2: 4.05061e+06
[10]	valid_0's l2: 3.52215e+06
[11]	valid_0's l2: 3.06983e+06
[12]	valid_0's l2: 2.68248e+06
[13]	valid_0's l2: 2.35573e+06
[14]	valid_0's l2: 2.07496e+06
[15]	valid_0's l2: 1.83742e+06
[16]	valid_0's l2: 1.63542e+06
[17]	valid_0's l2: 1.45845e+06
[18]	valid_0's l2: 1.31171e+06
[19]	valid_0's l2: 1.18266e+06
[20]	valid_0's l2: 1.07324e+06
[21]	valid_0's l2: 978148
[22]	valid_0's l2: 896838
[23]	valid_0's l2: 828028
[24]	valid_0's l2: 766981
[25]	valid_0's l2: 714706
[26]	valid_0's l2: 667653
[27]	valid_0's l2: 629270
[28]	valid_0's l2: 596524
[29]	valid_0's l2: 566841
[30]	valid_0's l2: 539817
[31]	valid_0's l2: 516530
[32]	valid_0's l2: 495382
[33]	valid_0's l2

In [156]:
gs.best_score_

-576.8761607121443

In [157]:
gs.best_estimator_

LGBMRegressor(learning_rate=0.08, max_bin=100, max_depth=16,
              min_data_in_leaf=10, num_leaves=45)

In [158]:
y_pred = gs.predict(X_test)

mean_squared_error(y_test, y_pred)**0.5

565.6889564608806

In [159]:
y_pred_val = gs.predict(X_val)

mean_squared_error(y_val, y_pred_val)**0.5

520.7735434618504

IDOLO

In [160]:
sub = pd.read_csv('../data/df_vol_predict.csv')


In [161]:
sub[cat_features] = sub.loc[:, cat_features].astype('category')

cat_df_sub = pd.get_dummies(sub[cat_features], drop_first = False)
num_df_sub = sub.loc[:, num_features]
X_sub = pd.concat([cat_df_sub, num_df_sub], axis = 1)

y_pred_sub = gs.predict(X_sub)

y_pred_sub.shape

(13485,)

In [162]:
submission = pd.DataFrame({'id': sub.index, 'price': y_pred_sub})

submission.to_csv('submission_new_1909_v4.csv', index=False)
